# End-To-End Example: Google Restaurants Map

Let's import our google functions and then use folium to plot restaurants on a map.



In [1]:
import requests 
import pandas as pd
import folium
import import_ipynb
import googleapi 
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

importing Jupyter notebook from googleapi.ipynb


In [2]:
google_key = '751838d6b9978955679461c80a7df60bde4f093d'

In [3]:
def geocode(location):
    '''
    using geocoding here: https://nominatim.openstreetmap.org/search
    '''
    query_string = {'q' : location, 'format' :'json'}
    url = 'https://nominatim.openstreetmap.org/search'
    response = requests.get(url, params = query_string)
    data = response.json()
    if type(data) == list:
        return data[0]
    else:
        return data
    

In [4]:
display(HTML("<h1>Yummy.com Restaurants</h1>"))
interact_zomato=interact.options(manual=True, manual_name="Find Eateries!")
@interact_zomato(cuisine="", city="")
def main(cuisine,city):
    geo = geocode(city)
    rests = googleapi.googlePlacesSearch(google_key, geo['lat'], geo['lon'], "restaurant", keyword=cuisine)
    rests_df = pd.json_normalize(rests)
    MAPCITY = (geo['lat'], geo['lon'])
    rmap = folium.Map(location=MAPCITY, zoom_start=10, width=800, height=600)
                    
    for row in rests_df.to_records():
        coords = (float(row['geometry.location.lat']),float(row['geometry.location.lng']))
        loc = f"<p><b>{row['name']}</b><br>{row['rating']}</p>"
        if row['rating'] > 4:
            color = "lightgreen"
        elif row['rating'] > 3:
            color = 'green'
        else:
            color = 'darkgreen'            
        marker = folium.Marker(location=coords, popup=loc,tooltip=loc, icon=folium.Icon(color=color))
        rmap.add_child(marker)
    display(rmap)

interactive(children=(Text(value='', continuous_update=False, description='cuisine'), Text(value='', continuou…